In [ ]:
!pip install faiss-gpu

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import faiss
from sklearn.model_selection import train_test_split as tts
import gc
from tqdm import tqdm

In [ ]:
train_data = np.load("/kaggle/input/title-embeddings/title_embeddings.npy")

In [ ]:
test_data = np.load("/kaggle/input/test-embs/test_embds.npy")

In [ ]:
target_labels = (pd.read_csv("/kaggle/input/aml-dataset/train.csv"))['PRODUCT_LENGTH'].values

In [ ]:
target_labels.shape

In [ ]:
x_train, x_val, y_train, y_val = tts(train_data, target_labels, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

In [ ]:
del train_data
gc.collect()

In [ ]:
del target_labels
gc.collect()

In [ ]:
y_train[0]

In [ ]:
cpu_index = faiss.IndexFlatL2(512)

In [ ]:
co = faiss.GpuMultipleClonerOptions()
co.shard = True
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index, co=co, ngpu=1)
gpu_index.add(x_train)

In [ ]:
gpu_index.train(x_train)

In [ ]:
x_val[:100].shape

In [ ]:
dist, indi = gpu_index.search(test_data, )

In [ ]:
indi[0]

In [ ]:
indi.shape

In [ ]:
np.mean(y_train[indi[0]])

In [ ]:
preds = []
for i in tqdm(range(indi.shape[0])):
    preds.append(np.mean(y_train[indi[i]]))

In [ ]:
len(preds)

In [ ]:
preds[:20]

In [ ]:
del test_data
gc.collect()

In [ ]:
ids = (pd.read_csv("/kaggle/input/aml-dataset/test.csv"))['PRODUCT_ID'].values
ids

In [ ]:
ids.shape

In [ ]:
d = {"PRODUCT_ID": ids, "PRODUCT_LENGTH": preds}

In [ ]:
df = pd.DataFrame(data=d)
df.head()

In [ ]:
df.to_csv("sub_f1.csv", index=False)